<a href="https://colab.research.google.com/github/psantul/Mtech_thesis_2022/blob/main/ANN_for_regression_using_elephas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install elephas==0.4.5

In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt update
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml import Pipeline


In [ ]:
# Create Spark context
conf = SparkConf().setAppName('BostonHousing_Spark_MLP').setMaster('local[*]')
sc = SparkContext(conf=conf)

ValueError: ignored

In [ ]:
pip install PyYAML==5.4.1

In [ ]:
# Define basic parameters
import pandas as pd
import numpy as np 

batch_size = 64
epochs = 10

# Load data
data= pd.read_csv("/content/drive/MyDrive/LALR.csv")
data[data["modelMag_u"]==-9999]=np.nan
data[data["modelMag_g"]==-9999]=np.nan
data[data["modelMag_r"]==-9999]=np.nan
data[data["modelMag_i"]==-9999]=np.nan
data[data["modelMag_z"]==-9999]=np.nan
data[data["fiberMag_u"]==-9999]=np.nan
data[data["fiberMag_g"]==-9999]=np.nan
data[data["fiberMag_r"]==-9999]=np.nan
data[data["fiberMag_i"]==-9999]=np.nan
data[data["fiberMag_z"]==-9999]=np.nan
data[data["dered_u"]==-9999]=np.nan
data[data["dered_g"]==-9999]=np.nan
data[data["dered_r"]==-9999]=np.nan
data[data["dered_i"]==-9999]=np.nan
data[data["dered_z"]==-9999]=np.nan
data[data["redshift"]<0]=np.nan
print(data.shape)
data=data.dropna()
print(data.shape)
data1=data.to_numpy()

(499998, 33)
(499732, 33)


In [ ]:
data

,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,modelMag_ug,modelMag_gr,modelMag_ri,modelMag_iz,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,fiberMag_ug,fiberMag_gr,fiberMag_ri,fiberMag_iz,petroR50_rr,petroR90_zz,ri,iz,dered_u,dered_g,dered_r,dered_i,dered_z,petroMag_uu,petroMag_gg,petroMag_rr,petroMag_ii,petroMag_zz,redshift
0,18.63367,18.32343,18.00390,17.90613,17.80887,0.31024,0.31953,0.09777,0.09726,18.97055,18.66509,18.37645,18.26491,18.17366,0.30546,0.28864,0.11154,0.09125,0.447172,0.427554,0.09777,0.09726,18.51204,18.22865,17.93834,17.85741,17.77264,18.546524,18.257280,17.991669,17.904271,17.813342,1.136832
1,20.42920,18.43875,19.02415,17.63169,23.19052,1.99045,-0.58540,1.39246,-5.55883,22.07500,20.48703,19.57536,19.04174,18.63651,1.58797,0.91167,0.53362,0.40523,0.643350,1.000000,1.39246,-5.55883,20.31271,18.34798,18.96136,17.58503,23.15581,21.083042,19.123444,19.595778,18.380089,22.874383,0.077724
2,19.09698,18.24340,17.99472,17.87978,17.78229,0.85358,0.24868,0.11494,0.09749,20.00031,19.17252,18.97012,18.84720,18.72353,0.82779,0.20240,0.12292,0.12367,0.400243,0.355879,0.11494,0.09749,18.99623,18.16489,17.94042,17.83943,17.75227,19.229591,18.341557,18.099712,17.927523,17.829662,0.023219
3,18.94601,17.41685,16.66702,16.26281,15.96437,1.52916,0.74983,0.40421,0.29844,20.16053,18.62279,17.86405,17.43543,17.13951,1.53774,0.75874,0.42862,0.29592,0.341635,0.341081,0.40421,0.29844,18.82164,17.31994,16.59999,16.21299,15.92731,18.749182,17.354763,16.696120,16.314472,16.100555,0.092937
4,17.47912,16.09549,15.40855,15.05322,14.76498,1.38363,0.68694,0.35533,0.28824,19.65367,18.07723,17.29426,16.87859,16.54164,1.57644,0.78297,0.41567,0.33695,0.448481,0.418961,0.35533,0.28824,17.36187,16.00413,15.34535,15.00626,14.73004,17.145967,15.937575,15.339691,15.004376,14.737408,0.045453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,18.99426,17.36810,16.58402,16.20523,15.86163,1.62616,0.78408,0.37879,0.34360,20.70968,19.04063,18.18647,17.77382,17.44293,1.66905,0.85416,0.41265,0.33089,0.406708,0.403354,0.37879,0.34360,18.58247,17.04723,16.36205,16.04028,15.73894,18.427698,17.188134,16.594786,16.303980,16.037207,0.062305
499994,26.30986,22.24602,20.79212,19.60717,18.87149,4.06384,1.45390,1.18495,0.73568,25.84696,22.92900,21.51387,20.28887,19.55563,2.91796,1.41513,1.22500,0.73324,0.369900,0.492835,1.18495,0.73568,26.02305,22.02254,20.63752,19.49229,18.78604,26.847554,22.048673,20.595540,19.353425,18.560887,0.580707
499995,17.64359,16.57409,15.94364,15.57373,15.35887,1.06950,0.63045,0.36991,0.21486,18.60426,17.62134,17.10917,16.74243,16.53295,0.98292,0.51217,0.36674,0.20948,0.295260,0.309870,0.36991,0.21486,17.36313,16.35556,15.79246,15.46139,15.27531,17.625218,16.402484,15.806505,15.462681,15.279260,0.044006
499996,21.18099,20.11874,19.95208,19.94004,19.65134,1.06225,0.16666,0.01204,0.28870,21.40167,20.44057,20.29401,20.26854,20.03115,0.96110,0.14656,0.02547,0.23739,0.452512,0.540297,0.01204,0.28870,20.90148,19.90096,19.80142,19.82808,19.56807,20.702186,19.947462,19.859656,19.906900,19.758372,2.723559


In [ ]:
#data.na.fill(value=0,subset=["modelMag_u","modelMag_g","modelMag_r","modelMag_i","modelMag_z","fiberMag_u","fiberMag_g","fiberMag_r","fiberMag_i","fiberMag_z","dered_u","dered_g","dered_r","dered_i","dered_z",]).show()
#data.na.fill(value=0).show()
#data.replace(nan, 0)

In [ ]:
print(data1)

[[18.63367    18.32343    18.0039     ... 17.90427082 17.81334188
   1.136832  ]
 [20.4292     18.43875    19.02415    ... 18.38008898 22.87438277
   0.07772421]
 [19.09698    18.2434     17.99472    ... 17.92752332 17.82966204
   0.02321945]
 ...
 [17.64359    16.57409    15.94364    ... 15.4626806  15.27926011
   0.04400648]
 [21.18099    20.11874    19.95208    ... 19.9069002  19.75837247
   2.723559  ]
 [20.01398    19.70229    19.28343    ... 19.0924852  19.02991704
   1.255538  ]]


In [ ]:
import numpy as np
data1 = np.delete(data1, 9, 1)
data1 = np.delete(data1, 17, 1)

In [ ]:
data1.shape

(499732, 31)

In [ ]:
X=data1[:,0:29]
#X=data1[:,1:19]
y=data1[:,-1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X)

# transform training data
X = norm.transform(X)

In [ ]:
print(y)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.33,random_state=42)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


model = Sequential()
model.add(Dense(64, input_shape=(x_train.shape[1],)))
model.add(Activation('sigmoid'))
model.add(Dense(128))
model.add(Activation('sigmoid'))
model.add(Activation('sigmoid'))
model.add(Dense(256))
model.add(Activation('sigmoid'))
model.add(Dense(1))
model.add(Activation('relu'))

[1.136832   0.07772421 0.02321945 ... 0.04400648 2.723559   1.255538  ]
334820 train samples
164912 test samples


In [ ]:

# Build RDD from numpy features and labels
df = to_data_frame(sc, x_train, y_train)
test_df = to_data_frame(sc, x_test, y_test)

#sgd_conf = optimizers.serialize(sgd)
opt = optimizers.Adam(lr=0.01)
sgd_conf = optimizers.serialize(opt)
# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("mae")
estimator.set_metrics(['mse'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(False)


ElephasEstimator_dd9accd23d0c

In [ ]:


# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(df)

# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(test_df)
pnl = prediction.select("label", "prediction")
pnl.show(10)

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = RegressionMetrics(prediction_and_label)
print(metrics.meanAbsoluteError)
print(metrics.rootMeanSquaredError)


>>> Fit model
>>> Synchronous training complete.
+-------------------+----------+
|              label|prediction|
+-------------------+----------+
|          0.3645389|       0.0|
| 1.7452560000000001|       0.0|
|           1.428886|       0.0|
|           1.496038|       0.0|
|          0.6393591|       0.0|
|0.12874839999999999|       0.0|
|          0.3162909|       0.0|
|           1.387032|       0.0|
|         0.06483295|       0.0|
|          0.2590229|       0.0|
+-------------------+----------+
only showing top 10 rows

0.6499384378686418
0.9840009414049212
